# Integration using libraries, and flow plots


## Integration with libraries

Let's start off with a typical multi-variable calculus contour plot.

In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Before we get into technical details, let's start this class off with a beautiful example of how integration is useful for visualization. 

In [ ]:
## a contour plot.  Let's try a typical multi-variable calculus
##  plot of level-sets and gradient vector field
fig, ax = plt.subplots() 
fig.set_size_inches(10,10) 

x,y = sp.symbols('x y')
## examples are the real parts of the family z^n = (x+iy)^n, n=2,3,4, etc.
#f = x*y
#f = x**3-x*y**2
f = x**4-6*x**2*y**2+y**4

grad = [sp.diff(f,x), sp.diff(f,y)]

Z = sp.lambdify( (x,y), f, "numpy")
Zx = sp.lambdify( (x,y), grad[0], "numpy")
Zy = sp.lambdify( (x,y), grad[1], "numpy")

Y, X = np.mgrid[-2:2:100j, -2:2:100j] ## x and y coordinates of a 2-dimension array of points
 ## in the plane

speed = np.sqrt(Zx(X,Y)**2 + Zy(X,Y)**2) ## is a 2-dimensional numpy array
 ## it is the speed of the gradient, at every 2-dimensional lattice point
mspd = speed.max()
    
CP = plt.contour(X,Y,Z(X,Y), 30) ## 30 contours plotted.
plt.clabel(CP, inline=1, fontsize=12) ## this add the level-set number to the curve

ax.streamplot(X, Y, Zx(X,Y), Zy(X,Y), color='0.2', density=0.5, 
              linewidth=10*(speed/mspd) )

plt.title("Level sets of $"+sp.latex(sp.Eq(sp.Symbol('f(x,y)'), f) )+"$\n"
          "and the gradient flow", fontsize=20)

The above plot has two primary devices:

 1) The **contour** plot, which we explored in [Homework assignment 2](../Homework/asst.2.ipynb).  In the above, we use it to plot the level-sets of the function
 
 $$ f(x,y) = xy.$$
 
 2) The **streamplot** plot.  In this case, we have computed the gradient vector field of $f$, 
 
 $$\nabla f(x,y) = (y, x)$$
 
 and the streamplot command plots a sampling of solutions to the differential equation
 
 $$\alpha'(t) = \nabla f(\alpha(t)).$$
 
 Streamplots are useful ways of visualizing vector fields. 

* * * 

### Today

We will explore the [**quad**](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.integrate.quad.html) command in the **scipy** library.  This is a command that automates numerical integration.

**Our goal**: To visualize vector fields in $\mathbb R^3$, like we did in $\mathbb R^2$ with the streamplot command. 

In [ ]:
from scipy import integrate as itg
t = sp.Symbol('t')

## integrate sin(t)
sp.pprint(sp.Integral(sp.sin(t), (t, 0, sp.pi)) )
print("Using **quad**")
print(itg.quad(np.sin, 0, np.pi),"\n" )

## integrate sin(t^2)
sp.pprint(sp.Integral(sp.sin(t**2), (t, 0, 12*sp.pi)) )
print("Using **quad**")
print(itg.quad(lambda x: np.sin(x**2), 0, 12*np.pi))

 * * *
 
**The output of quad** is a pair $(ANS, \epsilon)$ where $ANS$ is an approximation of the integral and $\epsilon$ is an upper bound on the error.  Precisely, if $(ANS,\epsilon)$ is the *quad* output for the integral
$$ \int_a^b f(t) dt$$
then it is known that $$\lvert \int_a^b f(t) dt - ANS \rvert < \epsilon$$
and the call to *quad* would be of the form **itg.quad(lambda t: f(t), (t, a, b) )**.

* * *

**Towards stream plots in $\mathbb R^3$**

* * *

**To make things concrete** let's set ourselves the goal of visualizing the magnetic field around a conducting loop of wire. 

**Background** The physical law that governs magnetic fields due to currents has the form

$$\vec B(\vec p) = c \frac{\vec v \times \hat r}{r^2} $$

* Here $\vec B$ is the magnetic field at a point $\vec p$. 
* $r$ is the distance between the point $\vec p$ and the moving charge, located at $\vec q$, $r = |\vec p - \vec q|$.   
* $\hat r$ is the unit vector from $\vec p$ to $\vec q$, i.e. $r \hat r = \vec q - \vec p$. 
* $\vec v$ is the velocity vector of the moving charge. 
* $\times$ is the cross-product of vectors in $\mathbb R^3$.
* $c$ is the magnitude of the charge at $\vec q$. 

Alternatively we could write this equation as
$$ \vec B = \frac{\vec I \times \hat r}{r^2}$$
where $\vec I = q \vec v$ is the *current vector*.

**Current in a wire**

Given a wire parametrized by a curve

$$\alpha : [a,b] \to \mathbb R^3$$

we could write the magnetic field at $\vec p$ as

$$\vec B(\vec p) = \int_a^b \frac{d\vec I \times (\alpha(t)-\vec p)}{r^3}$$

where $d\vec I = c(t) \alpha'(t) dt$, with $c(t)$ being the relative current density with respect to the $t$ parameter. Thus, if our curve was parametrized at constant speed and had uniform current density, then up to some multiple

$$\vec B(\vec p) = \int_a^b \frac{\alpha'(t) \times (\alpha(t)-\vec p)}{r^3} dt.$$

Let us compute these vector fields $\vec B$ in $\mathbb R^3$ for a few curves. 

* * *

**Problem 1**

Let's compute the vector fields $\vec B(\vec p)$ for a round circle. 

$$\alpha(t) = (\cos t, \sin t, 0)$$

so we want the integral 

 $$B(\vec p) = \int_0^{2\pi} \frac{(-\sin t, \cos t, 0)\times \left((\cos t, \sin t, 0) - \vec p\right)}{|(\cos t, \sin t, 0)-\vec p|^3}dt$$
 

At this stage we could, *in principle* determine the integrand *by hands* and give it as input to the *quad* command.  But that would involve a lot of algebra and we are too lazy.  So we use sympy to compute the integrand. 

In [ ]:
import sympy as sp
t,x,y,z = sp.symbols('t x y z')

num = sp.Matrix([-sp.sin(t), sp.cos(t), 0]).cross(\
      sp.Matrix([sp.cos(t)-x, sp.sin(t)-y, -z]))
num.simplify()

den = sp.Matrix([sp.cos(t)-x, sp.sin(t)-y, -z])
den = den.dot(den)**(3/2)

Fs = num/den
sp.pprint(Fs)
## our integrand, in closed form. 

In [ ]:
## Cast the above sympy expression into a list of
## callable functions. Sympy has a variety of ways of doing this. 
## We have seen the lambdify call.  We will use ufuncify instead, as
## this actually compiles the code and optimizes memory usage. 

from sympy.utilities.autowrap import ufuncify

F = []
for i in range(3):
    F.append(ufuncify([t,x,y,z], Fs[i,0]) )

In [ ]:
## While we're at it, we'll create 'vectorized' versions of our integration
## routines, to allow for some matlab-style notation.

def quad_1(f, I, *args):
    return np.vectorize(lambda n: itg.quad(f, I[0], I[1], (n,)+args)[0])

def f(x,k):
    return np.sin(x*k)

K = np.mgrid[0:1:6j]
print("K == ", K)
## integral of f(x,k)dx for various values of k.
print(quad_1(f, [0,1] )(K))

In [ ]:
## We shine things up a little below by creating a `vectorized' version of the quad integration
## routine. 

## quad_1 takes as input:
##  "f" a callable function
##   I a 2-element list giving the endpoints of integration
## and three additional arguments, which is a 3-dimensional array of points to call
## the quad integration on.  
##
## i.e. this routine allows for us to use the "quad" integration
## more like a numpy or matlab command. 
def quad_1(f, I, *args):
    return np.vectorize(lambda n,m,p: itg.quad(f, I[0], I[1], (n,m,p)+args)[0])

In [ ]:
## And this is a vector-valued version of quad_1. 
## this takes as input:
##  F : a list/tuple of single-variable functions on [a,b] with 3 additional parameters
##  I : I = [a,b] a 2-element list describing the domain of the primary parameter
##  returns a tuple of functions
def quad_n(F, I, *args):
    retval = []
    for f in F:
        retval.append([quad_1(f, I)(*args)])
    return tuple(retval)

x, y, z = np.mgrid[-1.5:1.5:8j, -1.5:1.5:8j, -0.5:0.5:6j]

u, v, w = quad_n(F, [0, 2*np.pi], x,y,z )

In [ ]:
import matplotlib as mpl
%matplotlib nbagg
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt

mpl.rcParams['legend.fontsize'] = 10

fig = plt.figure()
ax = fig.gca(projection='3d')

## plotting the circle
T = np.arange(0.0, 2*np.pi, 0.01)
ax.plot(np.cos(T), np.sin(T), np.zeros_like(T), 'r')

## vector field
q = ax.quiver(x, y, z, u, v, w, length=0.1, pivot='middle', cmap='Blues', linestyles='solid')

#q.set_array(np.random.rand(np.prod(x.shape)))
plt.show()

## quiver call details available in two locations.  The 3d plotting API
## http://matplotlib.org/mpl_toolkits/mplot3d/api.html
## and the linecollection API
## http://matplotlib.org/api/collections_api.html#matplotlib.collections.LineCollection

ax.set_xlabel('x-axis')
ax.set_ylabel('y-axis')
ax.set_zlabel('z-axis')

plt.show()

Before we move on to more sophisticated plots, let's do another example to demonstrate
how flexible our tools are.  We will compute the magnetic field around a conducting [(p,q)-torus knot](https://plot.ly/~Ryan.Budney/12/_72-torus-knot/). 

In [ ]:
import sympy as sp
t,x,y,z = sp.symbols('t x y z')
spp, spq, spr, spR = sp.symbols("p q r R", real=True)

c = sp.Matrix([(spR+spr*sp.cos(2*sp.pi*spq*t))*sp.cos(2*sp.pi*spp*t), 
     (spR+spr*sp.cos(2*sp.pi*spq*t))*sp.sin(2*sp.pi*spp*t), 
      spr*sp.sin(2*sp.pi*spq*t)])
cp = sp.diff(c, t) ## the derivative
sp.pprint(c)

In [ ]:
p,q = 2, 3
C = c.xreplace({spp: p, spq: q, spR: 1.2, spr: 0.4})
Cp = cp.xreplace({spp: p, spq: q, spR: 1.2, spr: 0.4})
P = sp.Matrix([x,y,z])

sp.pprint(Cp)


In [ ]:
num = Cp.cross(C-P)
num.simplify()
den = C - P
den = den.dot(den)**(3/2)
Fs = num/den
#sp.pprint(Fs)

F = []
for i in range(3):
    F.append(ufuncify([t,x,y,z], Fs[i,0]) )
    
Cc = [] ## the original curve
for i in range(3):
    Cc.append(ufuncify([t], C[i,0]))
    
x, y, z = np.mgrid[-1.7:1.7:5j, -1.7:1.7:5j, -0.5:0.5:3j]
u, v, w = quad_n(F, [0, 2*np.pi], x,y,z )

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')

T = np.arange(0.0, 2*np.pi, 0.01)

ax.plot(Cc[0](T), Cc[1](T), Cc[2](T), 'r')

quiv = ax.quiver(x, y, z, u, v, w, length=0.1, pivot='middle', cmap='Blues', linestyles='solid')
plt.show()

## quiver call details available in two locations.  The 3d plotting API
## http://matplotlib.org/mpl_toolkits/mplot3d/api.html
## and the linecollection API
## http://matplotlib.org/api/collections_api.html#matplotlib.collections.LineCollection

ax.set_xlabel('x-axis')
ax.set_ylabel('y-axis')
ax.set_zlabel('z-axis')

plt.show()

In [ ]:
## By-hands implementation of Basic flow lines.  

## We will take as input the initial vector field defined on the
## x,y,z meshgrid, and flow a small amount using an Euler method. 

## takes as input the same ingredients as before.  But for every x,y,z coordinate
## where we plot the vector fields, let's plot a small amount of the flow.  To 
## do this we will need to iterate over the x,y,z coordinates and perform a few
## steps of Euler's method. 

## function will return a list of flowlines.  An individual flowline will be
## a 3-element list [x,y,z] where x,y,z are lists of x-coordinates, y-coordinates,z-coordinates
## respectively for the individual flow line. 

def squad(F, I, *args):
    retval = []
    for f in F:
        retval.append(itg.quad(f, I[0], I[1], args)[0])
    return retval

import itertools as it

x,y,z=np.mgrid[-1.8:1.8:8j, -1.8:1.8:8j, -0.9:0.9:6j]

## returns the flow lines as a (list) of (list of triples) of (list of floats)
## takes as input a mgrid of x,y,z coordinates, number of steps n and time step dt
## Also requires F callable function of 3-variables: t,x,y,z. We integrate against t,
## from 0 to 2pi for now. Might want to make this more flexible in the future.
## C is the original curve parametrization, assumed domain is [0,2pi]
def flowLines(x,y,z,F,C,n,dt):
    def onC(x,y,z,Cc):
        I = np.arange(0,2*np.pi, 0.01)
        if min([np.sqrt((Cc[0](t)-x)**2 + (Cc[1](t)-y)**2 + (Cc[2](t)-z)**2) for t in I]) < 0.02:
            return True ## close enough
        else:
            return False ## it's safe to use Euler's method
    
    retval = []
    for i,j,k in it.product(range(x.shape[0]), range(x.shape[1]), range(x.shape[2])):
        ## okay, let's build the flowline at x,y,z
        ## determine vector field at x,y,z, flow, build flowline, repeat
        xl = [x[i,j,k]]; yl = [y[i,j,k]]; zl = [z[i,j,k]];
        if onC(x[i,j,k], y[i,j,k], z[i,j,k], C) == False:
            for s in range(n):
                ## one step of Euler's method
                #integrate Fdt at xl,yl,zl from 0 to 2pi.
                #print(xl[-1], yl[-1], zl[-1])
                dp = squad(F, [0, 2*np.pi], xl[-1], yl[-1], zl[-1])
                ldp = np.sqrt(dp[0]**2+dp[1]**2+dp[2]**2)
                if (ldp<1.0): ldp = 1.0
                xl.append( xl[-1] + dt*dp[0]/ldp )
                yl.append( yl[-1] + dt*dp[1]/ldp )
                zl.append( zl[-1] + dt*dp[2]/ldp )
            
        ## append the flowline to retal.
        retval.append([xl, yl, zl])
        
    return retval

flows = flowLines(x,y,z, F, Cc, 60, 0.005)

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')

for P in flows:
    ax.plot(P[0], P[1], P[2], 'r')

T = np.arange(0.0, 2*np.pi, 0.01)

ax.plot(Cc[0](T), Cc[1](T), Cc[2](T), 'b')

plt.show()

In [ ]:
## Let's see if we can do better with plot.ly

## on your virtual machine
## sudo pip install plotly

import sys
expaths = ["/usr/lib/python3/dist-packages", "/usr/local/lib/python3.5/dist-packages"]
for xp in expaths:
    if (xp not in sys.path):
        sys.path.append(xp)
import plotly.graph_objs as go
import plotly.plotly as py
import plotly.offline as pyoff
pyoff.init_notebook_mode(connected=True)

In [ ]:
T = np.arange(0,2*np.pi, 0.004)

tracelist = [go.Scatter3d( ## the knot
    x=Cc[0](T),    y=Cc[1](T),     z=Cc[2](T),
    mode='lines',
    line=go.Line(color='#2050FF', width=18)
    )]

tipsx = [P[0][-1] for P in flows]
tipsy = [P[1][-1] for P in flows]
tipsz = [P[2][-1] for P in flows]

tracelist.append(go.Scatter3d(
    x=tipsx, y=tipsy, z=tipsz, mode='markers',
    marker = dict(
        size=2, color = 'yellow') ) )

for P in flows:
    tracelist.append( go.Scatter3d(x=P[0], y=P[1], z=P[2], 
                                   mode='lines',
                                   line=go.Line(color='#FF4040', width=3)) )

data=go.Data(tracelist)

layout = go.Layout(
    title='Magnetic Field around a conducting '+str(p)+','+str(q)+'-torus knot',
    scene=dict(
        xaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        zaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        )
    )
)

fig = go.Figure(data=data, layout=layout)

# using py "plain plotly" will push the rendering to the plotly webpage
#py.plot(fig, filename='electric'+str(p)+str(q)+'torus.html')  #creates new page (large)

# using pyoff "plotly offline" will render locally, only. 
pyoff.iplot(fig) #puts inline